### Anastomosing demo overview

Anchors:
- `analog_image_generator.geologic_generators.generate_anastomosing`
- `analog_image_generator.stacked_channels.build_stacked_fluvial`
- `analog_image_generator.stats.compute_metrics`
- `analog_image_generator.reporting.build_reports`

Use this notebook to demonstrate the features listed below. Fill in the upcoming sections (presets, generator runs, stacked packages, metrics, reporting, and debugging hooks) before the professor review.


# Fluvial Anastomosing Anchors
Linking GEOLOGIC_RULES entries to helper implementations.


#### Demo checklist

- [ ] Parameter presets for branches/marsh/fans
- [ ] Single-belt generator demo with marsh/fan overlays
- [ ] Stacked package showing marsh-rich units
- [ ] Visualization (branch_channel, marsh, fan) + legend
- [ ] Metrics summary
- [ ] Reporting hook
- [ ] Debug controls


In [ ]:
from analog_image_generator import geologic_generators as gg

ANASTO_BASELINE = {
    "style": "anastomosing",
    "height": 256,
    "width": 256,
    "seed": 314,
    "branch_count": gg._ANASTO_DEFAULTS["branch_count"],
    "levee_width_px": gg._ANASTO_DEFAULTS["levee_width_px"],
    "levee_height_scale": gg._ANASTO_DEFAULTS["levee_height_scale"],
    "marsh_fraction": gg._ANASTO_DEFAULTS["marsh_fraction"],
    "fan_length_px": gg._ANASTO_DEFAULTS["fan_length_px"],
}

MARSH_HEAVY = {
    **ANASTO_BASELINE,
    "marsh_fraction": 0.65,
    "branch_count": 4,
}

STACKED_WETLAND = {
    **ANASTO_BASELINE,
    "mode": "stacked",
    "package_count": 2,
    "package_styles": ["anastomosing", "meandering"],
    "package_relief_px": 16,
    "package_erosion_depth_px": 10,
}

ANASTO_PRESETS = {
    "baseline": ANASTO_BASELINE,
    "marsh_heavy": MARSH_HEAVY,
    "stacked_wetland": STACKED_WETLAND,
}
ANASTO_PRESETS

In [ ]:
from analog_image_generator import geologic_generators as gg
from analog_image_generator import stacked_channels as sc


def run_single_anasto(params: dict) -> tuple:
    config = {**ANASTO_PRESETS['baseline'], **params}
    return gg.generate_fluvial(config)

def run_stacked_anasto(params: dict) -> tuple:
    config = {**ANASTO_PRESETS['stacked_wetland'], **params}
    return sc.build_stacked_fluvial(config)

single_anasto = run_single_anasto({})
stacked_anasto = run_stacked_anasto({})
single_anasto[0].shape, stacked_anasto[0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def _mask_arrays(masks_dict):
    arrays = []
    for name, value in masks_dict.items():
        if isinstance(value, np.ndarray) and value.ndim >= 2:
            arrays.append((name, value.astype(np.float32)))
    return arrays

def _plot_masks(analog, masks_dict, title: str):
    arrays = _mask_arrays(masks_dict)
    fig, axes = plt.subplots(1, len(arrays) + 1, figsize=(12, 4))
    axes[0].imshow(analog, cmap='gray', vmin=0.0, vmax=1.0)
    axes[0].set_title("Anasto (gray)")
    axes[0].axis('off')
    for ax, (name, mask) in zip(axes[1:], arrays):
        ax.imshow(mask, cmap='gray', vmin=0.0, vmax=1.0)
        ax.set_title(name)
        ax.axis('off')
    fig.tight_layout()

analog_single, masks_single = single_anasto
_plot_masks(analog_single, masks_single, 'Anasto single run')

analog_stacked, masks_stacked = stacked_anasto
fig = plt.figure(figsize=(6, 4))
plt.imshow(analog_stacked, cmap='gray', vmin=0.0, vmax=1.0)
package_map = masks_stacked.get('package_id_map')
if isinstance(package_map, np.ndarray) and package_map.ndim >= 2:
    plt.contour(package_map, levels=6, colors='r', linewidths=0.4)
plt.title('Anasto stacked overview')
plt.axis('off')
fig.tight_layout()

In [ ]:
import pandas as pd

from analog_image_generator import stats as ag_stats


def summarize_metrics(label: str, analog, masks):
    metrics = ag_stats.compute_metrics(analog, masks, env='fluvial')
    display(pd.DataFrame([metrics])[['beta_iso','entropy_global','fractal_dimension','psd_aspect','qa_psd_anisotropy_warning','qa_channel_area_warning']])
    print(f"Metrics recorded for {label}")
    return metrics

single_anasto_metrics = summarize_metrics('single', *single_anasto)
stacked_anasto_metrics = summarize_metrics('stacked', *stacked_anasto)

## anchor-fluvial-anasto-paths
`anasto_paths(H, W, branch_count, rng)` lays out narrow branches.

## anchor-fluvial-anasto-levees
`add_levees_narrow(branch_channel, width_px, height_scale)` builds thin levees.

## anchor-fluvial-anasto-marsh
`make_marsh(branch_channel, marsh_fraction, rng, shape)` derives marsh/overbank masks.

## anchor-fluvial-anasto-fans
`seed_fans(breach_points, fan_length_px, rng, shape)` draws crevasse fans.

## anchor-fluvial-anasto-compose
`compose_anasto(gray, masks, noise_scale, rng)` normalizes grayscale output.

## anchor-fluvial-anasto-generate
`generate_anastomosing(params, rng)` orchestrates branches, levees, marshes, fans, and metadata.

## anchor-sedimentary-approximations
Sedimentary overlays in this notebook use simplified distance-to-mask gradients, Gaussian-smoothed noise, and deterministic laminae frequencies (no fully resolved laminae or grain-scale physics). Mineralogy percentages are constrained heuristics (`feldspar`, `quartz`, `clay`) summing to 1.0; cement signatures toggle between kaolinite/calcite based on marsh coverage, and mud_clasts_bool reflects overbank fraction. These approximations target qualitative realism for previews/statistics rather than exact petrophysical fidelity.


### anchor-fluvial-stack-relief

`analog_image_generator.stacked_channels.apply_relief_slice` lifts each package by the requested thickness/relief and `cut_erosional_surface` trims older stacks before the next deposit, 
producing the boundary masks that feed stacked stats. Use these helpers whenever stacked previews or QA notebooks highlight erosional relief assumptions.

### anchor-fluvial-anasto-compose

Automated anchor placeholder for GEOLOGIC_RULES mapping.

### anchor-fluvial-anasto-fans

Automated anchor placeholder for GEOLOGIC_RULES mapping.

### anchor-fluvial-anasto-generate

Automated anchor placeholder for GEOLOGIC_RULES mapping.

### anchor-fluvial-anasto-levees

Automated anchor placeholder for GEOLOGIC_RULES mapping.

### anchor-fluvial-anasto-marsh

Automated anchor placeholder for GEOLOGIC_RULES mapping.

### anchor-fluvial-anasto-paths

Automated anchor placeholder for GEOLOGIC_RULES mapping.

### Reporting hook
- Reporting artifacts (CSV, env PDFs, master PDF) are generated via `python scripts/smoke_test.py`.
- Latest outputs live under `outputs/smoke_report/` and can be attached to demos.
- Use `analog_image_generator.reporting.build_reports` with the metrics rows captured above to regenerate per-env summaries if needed.

In [ ]:
import json
from pathlib import Path

TASK_TAG = "fluvial-v1-demo"


def _resolve_taskmaster_tasks() -> Path:
    search_roots = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for root in search_roots:
        candidate = root / '.taskmaster' / 'tasks' / 'tasks.json'
        if candidate.exists():
            return candidate
    raise FileNotFoundError(
        "Unable to locate .taskmaster/tasks/tasks.json relative to the notebook "
        "execution directory."
    )


TASKS_FILE = _resolve_taskmaster_tasks()


def override_seed(seed: int = 0, preset_key: str = 'baseline'):
    params = {**ANASTO_PRESETS[preset_key], 'seed': seed}
    analog, masks = gg.generate_fluvial(params)
    print(f"Generated preview for seed={seed} preset={preset_key}")
    return analog, masks


tasks_data = json.loads(TASKS_FILE.read_text())
tasks_snapshot = tasks_data.get(TASK_TAG, {}).get('tasks', [])
tasks_snapshot[:3]


### Debug + seed override controls (Anastomosing)
These controls expose Task Master-ready hooks for overriding seeds, switching between single-belt and stacked packages, and surfacing QA metadata directly inside the notebook. Use them before demos so we can capture screenshots/logs without editing source files.


In [ ]:
# Debug + override helper functions for Anastomosing realizations
import json
from pprint import pprint

from IPython.display import Markdown, display

try:
    import ipywidgets as widgets
except ImportError:  # pragma: no cover - executed in notebook
    widgets = None

STYLE_LABEL = "Anastomosing"

OVERRIDE_STATE = {
    "preset": "baseline",
    "stack_mode": "single",
    "seed": ANASTO_PRESETS["baseline"]["seed"],
}
DEBUG_FLAGS = {
    "show_metadata_table": True,
    "show_debug_accordion": True,
}


def run_fluvial_override(preset_key: str, stack_mode: str, seed: int):
    '''Run a single/stacked realization using the requested overrides.'''
    params = {**ANASTO_PRESETS[preset_key]}
    params["seed"] = seed
    params["_preset_label"] = preset_key
    params["mode"] = stack_mode if stack_mode == "stacked" else "single"
    if stack_mode == "stacked":
        analog, masks = sc.build_stacked_fluvial(params)
    else:
        analog, masks = gg.generate_fluvial(params)
    return analog, masks, params


def render_metadata_table(masks: dict, params: dict):
    '''Display summary + stacked package metadata for Task Master logging.'''
    metadata = masks.get("realization_metadata", {})
    summary = {
        "preset": params.get("_preset_label", "baseline"),
        "stack_mode": params.get("mode", "single"),
        "seed": params.get("seed"),
        "package_count": metadata.get("package_count"),
        "erosion_relief_px": metadata.get("erosion_relief_px"),
        "qa_flags": metadata.get("qa_flags"),
    }
    display(pd.DataFrame([summary]))
    packages = metadata.get("stacked_packages")
    if packages:
        display(pd.DataFrame(packages))
    print("Metadata JSON payload:")
    print(json.dumps(metadata, indent=2))


def build_debug_accordion(analog, masks, params):
    '''Use ipywidgets.Accordion to reveal previews + parameter dictionaries on demand.'''
    if widgets is None:
        print("ipywidgets unavailable; skipping accordion output.")
        return
    preview_output = widgets.Output()
    with preview_output:
        _plot_masks(analog, masks, f"{STYLE_LABEL} override preview")
    params_output = widgets.Output()
    with params_output:
        pprint(params)
    metadata_output = widgets.Output()
    with metadata_output:
        metadata = masks.get("realization_metadata", {})
        print(json.dumps(metadata, indent=2))
    accordion = widgets.Accordion(children=[preview_output, params_output, metadata_output])
    accordion.set_title(0, "Raw mask preview")
    accordion.set_title(1, "Parameter dictionary")
    accordion.set_title(2, "Metadata JSON")
    display(accordion)


In [ ]:
# Interactive override runner for Anastomosing demos
if widgets is None:
    print(
        "ipywidgets is not installed; adjust OVERRIDE_STATE and call "
        "run_fluvial_override(**OVERRIDE_STATE) manually."
    )
else:
    preset_dropdown = widgets.Dropdown(
        options=sorted(ANASTO_PRESETS.keys()),
        value=OVERRIDE_STATE["preset"],
        description="Preset",
    )
    stack_toggle = widgets.ToggleButtons(
        options=[("Single", "single"), ("Stacked", "stacked")],
        value=OVERRIDE_STATE["stack_mode"],
        description="Mode",
    )
    seed_box = widgets.IntText(value=OVERRIDE_STATE["seed"], description="Seed")
    debug_toggle = widgets.Checkbox(
        value=DEBUG_FLAGS["show_debug_accordion"],
        description="Show accordion",
    )
    output_area = widgets.Output()

    def _run_override(_=None):
        OVERRIDE_STATE.update(
            {
                "preset": preset_dropdown.value,
                "stack_mode": stack_toggle.value,
                "seed": seed_box.value,
            }
        )
        analog, masks, params = run_fluvial_override(
            OVERRIDE_STATE["preset"],
            OVERRIDE_STATE["stack_mode"],
            OVERRIDE_STATE["seed"],
        )
        with output_area:
            output_area.clear_output()
            display(
                Markdown(
                    f"**Override run** → preset `{OVERRIDE_STATE['preset']}`, "
                    f"seed `{OVERRIDE_STATE['seed']}`, "
                    f"mode `{OVERRIDE_STATE['stack_mode']}`"
                )
            )
            render_metadata_table(masks, params)
            if debug_toggle.value:
                build_debug_accordion(analog, masks, params)

    run_button = widgets.Button(description="Run override", button_style="primary")
    run_button.on_click(_run_override)
    display(
        widgets.VBox(
            [
                widgets.HBox([preset_dropdown, stack_toggle, seed_box]),
                debug_toggle,
                run_button,
                output_area,
            ]
        )
    )
    _run_override()


### Debug + QA checklist (Anastomosing)
- [ ] Captured a seed override + mode toggle screenshot for Task Master notes
- [ ] Logged metadata table output into `.taskmaster/tasks/tasks.json` (Task 7 subtask 3)
- [ ] Expanded the debug accordion to verify raw masks + parameter dictionaries
- [ ] Re-ran `python scripts/validate_geo_anchors.py` and `python scripts/smoke_test.py`
- [ ] Checked this notebook in Git with updated outputs after verifying overrides
